## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='my-cool-experiment', tags={}>]

In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [3]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

In [4]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 0385f607dc8e40249af1c2da7515da6c, rmse: 6.3040
run id: fd79c09521524222b55ddcb5eb7f5bc6, rmse: 6.7423
run id: 04766fc937d441b2a2bea1cbdfb057d1, rmse: 6.9074


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [32]:
run_id = "0385f607dc8e40249af1c2da7515da6c"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-reg")

Registered model 'nyc-reg' already exists. Creating a new version of this model...
2022/07/17 20:44:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-reg, version 8
Created version '8' of model 'nyc-reg'.


<ModelVersion: creation_timestamp=1658087053314, current_stage='None', description=None, last_updated_timestamp=1658087053314, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=8>

In [33]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1658085338952, description='', last_updated_timestamp=1658087053314, latest_versions=[<ModelVersion: creation_timestamp=1658086687484, current_stage='Staging', description=None, last_updated_timestamp=1658086893019, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=7>,
  <ModelVersion: creation_timestamp=1658086618533, current_stage='Archived', description=None, last_updated_timestamp=1658087039074, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>,
  <ModelVersion: creation_timestamp=1658087053314, current_stage='None', description=None, last_updated_timestamp=1658087053314, name='nyc-reg', run_id='0385f607dc8e40249af1

In [30]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1658085338952, description='', last_updated_timestamp=1658086893019, latest_versions=[<ModelVersion: creation_timestamp=1658086687484, current_stage='Staging', description=None, last_updated_timestamp=1658086893019, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=7>,
  <ModelVersion: creation_timestamp=1658086612126, current_stage='Archived', description=None, last_updated_timestamp=1658086820504, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=5>,
  <ModelVersion: creation_timestamp=1658086618533, current_stage='Production', description=None, last_updated_timestamp=1658086868668, name='nyc-reg', run_id='0385f607dc8e4

In [42]:
model_name = "nyc-reg"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}, source: {version.source}")

version: 8, stage: Staging, source: ./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow
version: 6, stage: Archived, source: ./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/model
version: 7, stage: Production, source: ./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow


In [41]:
model_version = 8
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1658087053314, current_stage='Staging', description='The model version 8 was transitioned to Staging on 2022-07-17', last_updated_timestamp=1658087107453, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=8>

In [43]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1658087053314, current_stage='Staging', description='The model version 8 was transitioned to Staging on 2022-07-17', last_updated_timestamp=1658087110615, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=8>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [12]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'].astype('str') + '_' + df['DOLocationID'].astype('str')
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [13]:
df = read_dataframe("data/green_tripdata_2021-02.parquet")

In [14]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/mohammad/Documents/local/tutorial_env/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [15]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [16]:
X_test = preprocess(df, dv)

In [17]:
target = "duration"
y_test = df[target].values

In [44]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[20:45:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 18 s, sys: 92.7 ms, total: 18 s
Wall time: 5.22 s


{'rmse': 6.303975653496241}

In [45]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[20:45:20] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 18.8 s, sys: 169 ms, total: 18.9 s
Wall time: 6.38 s


{'rmse': 6.303975653496241}

In [60]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1658086160134, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-07-17', last_updated_timestamp=1658086287804, name='nyc-reg', run_id='0385f607dc8e40249af1c2da7515da6c', run_link=None, source='./mlruns/1/0385f607dc8e40249af1c2da7515da6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=3>